# 프로젝트: KoChatGPT 업그레이드 하기

## import modules

In [22]:
import pandas as pd
import numpy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

# 1. 기존 KoGPT2와 SFT 적용 모델 결과 분석

### 정량적 분석을 위한 bleu 평가 함수 만들기

In [57]:
!pip install sacrebleu --q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [58]:
from datasets import load_metric

# Load the BLEU metric
metric = load_metric("sacrebleu")

In [59]:
# Function to tokenize a sentence
def tokenize_sentence(sentence):
    # Implement tokenization using your tokenizer
    # Return a list of tokens
    tokens = tokenizer.tokenize(sentence)
    return tokens

In [60]:
# Function to calculate BLEU score
def calculate_bleu_score(model_outputs, target_sentences):
    # Tokenize model outputs and target sentences
    tokenized_outputs = [tokenize_sentence(output) for output in model_outputs]
    tokenized_targets = [tokenize_sentence(target) for target in target_sentences]
    
    # Calculate BLEU score
    bleu_score = metric.compute(predictions=tokenized_outputs, references=tokenized_targets)
    
    return bleu_score

### 평가를 위한 데이터셋 만들기 (효율을 위해 데이터셋 크기 줄이기 : 100개)

In [61]:
data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file_SFT:
    list_data_dict_SFT = json.load(json_file_SFT)[11900:]

print(len(list_data_dict_SFT))
list_data_dict_SFT[:3]

100


[{'prompt': '플랫 화이트가 뭔가요?',
  'completion': '\'"플랫 화이트"는 일반적으로 흰색 배경에 만화나 일러스트 등을 담은 디자인을 의미합니다. 주로 웹툰, 게임, 애니메이션, 영화 등에서 사용되며, 평면적으로 표현된 그래픽 디자인의 스타일을 말합니다.',
  'tokens': 127},
 {'prompt': '어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가?',
  'completion': "'김영삼의 서울대학교 입학 및 졸업 증명서는 서울대학교 미술관에 전시되어 있습니다.",
  'tokens': 105},
 {'prompt': '제가 지금 강남역인데, 여기서 오래 걸릴까요?',
  'completion': "'저는 인공지능 챗봇이므로 현재의 교통 상황과 여행 거리, 이동 방법 등과 같은 정보를 보유하고 있지 않습니다. 하지만, 일반적으로 강남역은 교통이 굉장히 복잡하고 혼잡하기 때문에 시간이 걸릴 수 있습니다. 만약 이동하실 위치가 어디인지 알려주신다면, 보다 더 정확한 답변을 드릴 수 있을 것입니다.",
  'tokens': 195}]

## 기존 KoGPT2 모델 결과 및 평가

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
input_prompt = []
for i in list_data_dict_SFT:
    input_prompt.append(i['prompt'])
    
labeled_completion = []
for i in list_data_dict_SFT:
    labeled_completion.append(i['completion'])

In [9]:
initial_output = []
max_length = 128

for input_txt in input_prompt:
    input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
    output= model.generate(input_ids, max_length=max_length, num_beams=4, no_repeat_ngram_size=4, early_stopping=True,
                                 eos_token_id=375, do_sample=True, top_k=50, repetition_penalty=2.0)
    initial_output.append(tokenizer.decode(output[0]))

In [13]:
initial_output[:3]

['플랫 화이트가 뭔가요?"\n"그럼, 그건 뭐죠?"\n"네, 그렇군요."\n"아무래도 저는 좀 이상하다고 생각했는데."\n"어떻게 생각해보세요?"\n나는 고개를 갸웃거렸다.\n"다행히도 어젯밤에는 그런 일이 없었어요."\n"그래요, 그럼, 오늘 밤은 어떻게 해볼까요?"\n내가 대답하자 그녀는 나를 쳐다보았다.\n"내일 아침에 다시 한 번 말씀드릴게요."\n"이번에는 아무래도 좋지 않을 것 같아요."\n"왜냐하면 이번에야말로 당신 혼자',
 '어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가?,\n',
 '제가 지금 강남역인데, 여기서 오래 걸릴까요?"\n"그럼, 그건 아니겠지."\n"네, 그렇습니다. 저도 그렇게 생각합니다."\n"그래요, 그럴 수도 있지요."\n"아닙니다. 그런 생각을 해본 적이 없습니다."\n"그렇다면 당신은 무슨 생각을 하고 계십니까?"\n"글쎄요, 그렇지 않습니다."\n"물론이지요. 하지만 말입니다."\n"당신이 뭐라고 했습니까?"\n"말씀드릴 수 없습니다. 말씀드릴 수 있는 것은 아무것도 없습니다."\n이번에는 고개를 끄덕였다.\n"어떻게 된 일입니까?"\n']

### bleu score 평가

In [11]:
# Calculate BLEU score
initial_bleu_score = []
for output, target in zip(initial_output, labeled_completion):
    initial_bleu_score.append(calculate_bleu_score([output], [target])['score'])
    
sum(initial_bleu_score)/len(initial_bleu_score)

0.22749754512908654

## 기존 모델의 bleu score : 0.227

In [16]:
del model
torch.cuda.empty_cache()

## SFT 적용 모델 결과 및 평가

In [56]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [18]:
### 모델과 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=128,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=128, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [22]:
### 모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의
from typing import Optional, Dict, Sequence

class SFT_dataset_train(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset_train, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [23]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [24]:
### SFT_dataset 클래스를 사용해 훈련셋 만들기 + data collator 인스턴스 만들기
train_dataset = SFT_dataset_train(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [26]:
### Training arguments를 사용해 trainer 클래스를 정의

training_args = TrainingArguments(
    output_dir="/aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    warmup_steps=15,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [28]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

Step,Training Loss
500,2.491600
1000,2.502200
1500,2.472300
2000,1.948200
2500,1.989400
3000,2.002100
3500,1.562800
4000,1.603700
4500,1.600400
5000,1.301300


In [31]:
SFT_output = []
max_length = 128

for input_txt in input_prompt:
    input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
    output= model.generate(input_ids, max_length=max_length, num_beams=4, no_repeat_ngram_size=4, early_stopping=True,
                                 eos_token_id=375, do_sample=True, top_k=50, repetition_penalty=2.0)
    SFT_output.append(tokenizer.decode(output[0]))

In [32]:
SFT_output[:3]

['플랫 화이트가 뭔가요?\\n\\n저는 AI 어시스턴트이기 때문에 개인적인 정보를 가지고 있지 않습니다. 따라서 해당 질문에 대한 답변을 드릴 수 없습니다. 죄송합니다!</s>ミン<unk>ー(어도비)입니다.</s>ミツアー<unk>(어도비)는 일본어로 "어디 있나요?"를 의미할 수 있습니다.</s>美ツアー(어도비는 일본어로 "옷" 또는 "갑옷"을 뜻합니다.</s>フ<unk><unk>・ジ<unk>ン(어도비)은 영어권에서 사용되는 용어 중 하나입니다.</s>英書(어도비)에 따르면,</s>',
 '어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가?\\n\\n김영삼 대통령의 서울대학교 입시에 대한 구체적인 정보가 제공되지 않아 정확한 답변을 제공할 수 없습니다. 따라서 해당 질문에 대한 답변을 제공해 드릴 수 없습니다. 추가적인 정보를 제공하시면 보다 정확한 답변을 제공해드리겠습니다.</s> emotions.</s> question={\'prompt\'pt\': "김영삼 대학교(서울대학교) 입학 및 졸업증명서(서울대학교)에 대한 세부 정보는 제공되지 않았습니다.", \'token\': 159}</s> isconsidered information',
 '제가 지금 강남역인데, 여기서 오래 걸릴까요?\\n\\n저는 인공지능 챗봇이기 때문에 시간을 측정할 수 없습니다. 하지만 일반적으로 강남역은 교통이 굉장히 중요한 역할을 하기 때문에, 혼잡하고 혼잡할 수 있는 지역을 피하고 빠른 시일 내에 빠르게 이동할 수 있도록 도와줍니다. 또한, 지하철 9호선이나 9호선 등 다른 교통 수단을 이용하여 이동하실 수도 있습니다. 따라서, 항상 혼잡하지 않은 지역을 선택하시고 빠른 시간 내에 이동하시는 것이 좋습니다.</s> 챗봇 이기백 선생님, 감사합니다.</s>홍영식 선생님, yooung-jin0109}</s>']

### bleu score 평가

In [33]:
# Calculate BLEU score
SFT_bleu_score = []
for output, target in zip(SFT_output, labeled_completion):
    SFT_bleu_score.append(calculate_bleu_score([output], [target])['score'])

sum(SFT_bleu_score)/len(SFT_bleu_score)

Token indices sequence length is longer than the specified maximum sequence length for this model (136 > 128). Running this sequence through the model will result in indexing errors


0.13761214766932256

## SFT 모델 bleu score : 0.137

### 답변 비교

In [46]:
input_prompt[5]

'고아인 숙희를 키워준 복순의 직업은?'

In [47]:
initial_output[5]

'고아인 숙희를 키워준 복순의 직업은?"\n"아니오. 그건 그렇고요. 저도 모르는 일이죠."\n"그렇다면 어떻게 해야 할까요?"\n"예. 알고 있습니다."\n"알았습니다. 그럼 이만 가보겠습니다."\n숙희의 목소리는 떨떠름했다.\n"이제부터 제가 하는 일들이 다 끝난 것 같군요."\n"그래, 그렇게 하세요."\n"왜 그러세요? 왜 이러는 거죠?"\n"어떻게 해야 할지 몰라서 그래요."\n"그러지 마세요."\n숙희는 한숨을 내쉬었다.\n"'

In [48]:
SFT_output[5]

'고아인 숙희를 키워준 복순의 직업은?\\n\\n저는 인공지능 언어모델로써 답변을 생성하는 AI이므로 복순의 직업에 대한 정보를 가지고 있지 않습니다. 하지만 귀하의 질문에 대한 답변은 귀하에게 직접 물어보는 것이 가장 정확할 수 있습니다. 예를 들어, 귀하가 귀하에게 어떤 일을 맡겼는지에 대한 정보가 있다면, 이를 바탕으로 정확한 답변을 제공할 수 있습니다.</s> 귀하: 귀하의 질문에서 언급된 "복순의 직업"에 대한 정보가 충분하지 않아 정확한 답변을 제공하기 어렵습니다. 추가적인 정보를 제공해주시면 보다 정확한 답변을 제공해드릴 수 있을 것입니다.</s> 귀하'

## 1. 기존 모델의 결과물과 SFT를 적용한 모델의 결과물을 정량/정성적으로 비교/분석
- 정량적 평가

    기존 모델의 bleu score : 0.227
    
    SFT 적용 모델의 bleu score : 0.137
    
    * bleu score는 기계 번역 결과와 정답이 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법이다.
    
        bleu는 precision을 기반으로 성능을 측정한다. 번역 시 정답 문장의 몇 단어가 빠져도 (즉 재현율이 떨어져도) 문장의 의미가 유사할 수 있지만, 문장에 없는 단어가 들어오게 되면 (정밀도가 떨어지면) 영향이 클 수 있기 때문이다.

        또한 문장 평가 시, n-gram의 n값에 따라 문장을 나눠서 정답과 얼마나 겹치는지 다각도로 평가한다. 그리고 n-gram 평가 방식을 사용할 때, 단어들의 일치도가 높더라도 정답 문장과 연속적으로 단어가 일치하지 않으면 bleu score가 0이 나올 수 있다. 

        따라서 bleu는 동일 단어의 빈도수가 중요하다. 
        
        현재 두 모델 모두 output에 input의 prompt가 포함되었기 때문에 실제보다 더 높은 score가 나올 가능성이 있다. SFT 모델의 bleu score가 더 낮게 나왔지만, 이는 정확하지 않은 지표라는 의문이 든다. 
        
        
        
- 정성적 평가

    기존 모델의 평가 : 기존의 모델은 질문자의 질문과 답변 간의 관계가 거의 없다. 질문자의 질문에 답변을 한다는 느낌보다는 비슷한 단어를 사용하여 답을 이어가는 느낌이었다. 하지만, 단어를 나열하는 순서나 문장의 완성도는 높았다.
    
    SFT 모델의 평가 : 질문자의 질문에 관련성 있게 답변을 하였다. 질문자의 질문을 정확히 이해하고 맥락에 맞게 답변하였다. 다만, 대부분 질문에 대해서 정확하게 답변을 제안하지 못했다.


    